# Chicago Car Crash Analysis

## Importing Packages

In [140]:
# Importing packages needed for this notebook

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [141]:
# Reading the data in

df_crashes = pd.read_csv("data/Traffic_Crashes_-_Crashes_20231109.csv")
df_vehicles = pd.read_csv("data/Traffic_Crashes_-_Vehicles_20231109.csv")
df_people = pd.read_csv("data/Traffic_Crashes_-_People_20231109.csv")

D:\Documents\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
D:\Documents\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (20,24,25,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [142]:
# Dropping columns

columns_to_drop_crashes = ['RD_NO', 'CRASH_DATE_EST_I', 'LANE_CNT', 'REPORT_TYPE', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DATE_POLICE_NOTIFIED',
                           'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I',
                           'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
                           'INJURIES_REPORTED_NOT_EVIDENT','INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'LATITUDE', 'LONGITUDE', 'LOCATION'
                           ]

columns_to_drop_vehicles = ['RD_NO', 'CRASH_DATE', 'CRASH_UNIT_ID', 'UNIT_NO', 'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE', 'MODEL', 'VEHICLE_YEAR',
                            'VEHICLE_DEFECT','VEHICLE_USE', 'TRAVEL_DIRECTION', 'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY', 'TOWED_TO',
                            'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I',
                            'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I',  'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO', 'COMMERCIAL_SRC',
                            'GVWR', 'CARRIER_NAME', 'CARRIER_STATE', 'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME','UN_NO', 'HAZMAT_PRESENT_I',
                            'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO', 'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I', 'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO',
                            'WIDE_LOAD_I', 'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 'TRAILER1_LENGTH', 'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH', 'AXLE_CNT', 'VEHICLE_CONFIG', 'CARGO_BODY_TYPE',
                            'LOAD_TYPE', 'HAZMAT_OUT_OF_SERVICE_I', 'MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS'
                            ]

columns_to_drop_people = ['RD_NO', 'CRASH_DATE', 'VEHICLE_ID', 'PERSON_ID', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'EJECTION',
                          'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'BAC_RESULT VALUE', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'CELL_PHONE_USE', 'INJURY_CLASSIFICATION'
                          ]

df_crashes = df_crashes.drop(columns_to_drop_crashes, axis=1)
df_vehicles = df_vehicles.drop(columns_to_drop_vehicles, axis=1)
df_people = df_people.drop(columns_to_drop_people, axis=1)

## Creating a Master DataFrame

In [143]:
master_df = pd.merge(df_crashes, pd.merge(df_vehicles, df_people, on="CRASH_RECORD_ID"), on="CRASH_RECORD_ID")

## Info and column names of each table

In [144]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3594423 entries, 0 to 3594422
Data columns (total 34 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CRASH_RECORD_ID          object 
 1   CRASH_DATE               object 
 2   POSTED_SPEED_LIMIT       int64  
 3   TRAFFIC_CONTROL_DEVICE   object 
 4   DEVICE_CONDITION         object 
 5   WEATHER_CONDITION        object 
 6   LIGHTING_CONDITION       object 
 7   FIRST_CRASH_TYPE         object 
 8   TRAFFICWAY_TYPE          object 
 9   ALIGNMENT                object 
 10  ROADWAY_SURFACE_COND     object 
 11  ROAD_DEFECT              object 
 12  CRASH_TYPE               object 
 13  DAMAGE                   object 
 14  PRIM_CONTRIBUTORY_CAUSE  object 
 15  NUM_UNITS                int64  
 16  MOST_SEVERE_INJURY       object 
 17  INJURIES_TOTAL           float64
 18  CRASH_HOUR               int64  
 19  CRASH_DAY_OF_WEEK        int64  
 20  CRASH_MONTH              int64  
 21  LIC_PLAT

## Takeaway

- 3,594,423 observations
- 36 combined columns
- 987 MB combined memory file

## Data Cleaning

In [145]:
# Filtering from 2020 and on

master_df['CRASH_DATE'] = pd.to_datetime(master_df['CRASH_DATE'])
master_df['YEAR'] = master_df['CRASH_DATE'].dt.year
master_df = master_df.loc[master_df['INJURIES_TOTAL'] > 0]
master_df = master_df[
    (master_df["YEAR"] >= 2020) &
    ((master_df["VEHICLE_TYPE"] == "PASSENGER") |
     (master_df["VEHICLE_TYPE"] == "SPORT UTILITY VEHICLE (SUV)") |
     (master_df["VEHICLE_TYPE"] == "VAN/MINI-VAN"))
]
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305598 entries, 0 to 3594374
Data columns (total 35 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          305598 non-null  object        
 1   CRASH_DATE               305598 non-null  datetime64[ns]
 2   POSTED_SPEED_LIMIT       305598 non-null  int64         
 3   TRAFFIC_CONTROL_DEVICE   305598 non-null  object        
 4   DEVICE_CONDITION         305598 non-null  object        
 5   WEATHER_CONDITION        305598 non-null  object        
 6   LIGHTING_CONDITION       305598 non-null  object        
 7   FIRST_CRASH_TYPE         305598 non-null  object        
 8   TRAFFICWAY_TYPE          305598 non-null  object        
 9   ALIGNMENT                305598 non-null  object        
 10  ROADWAY_SURFACE_COND     305598 non-null  object        
 11  ROAD_DEFECT              305598 non-null  object        
 12  CRASH_TYPE     

## Takeaway

- We filtered the data from year 2020 and after, 

In [146]:
master_df.isna().sum()

CRASH_RECORD_ID                0
CRASH_DATE                     0
POSTED_SPEED_LIMIT             0
TRAFFIC_CONTROL_DEVICE         0
DEVICE_CONDITION               0
WEATHER_CONDITION              0
LIGHTING_CONDITION             0
FIRST_CRASH_TYPE               0
TRAFFICWAY_TYPE                0
ALIGNMENT                      0
ROADWAY_SURFACE_COND           0
ROAD_DEFECT                    0
CRASH_TYPE                     0
DAMAGE                         0
PRIM_CONTRIBUTORY_CAUSE        0
NUM_UNITS                      0
MOST_SEVERE_INJURY             0
INJURIES_TOTAL                 0
CRASH_HOUR                     0
CRASH_DAY_OF_WEEK              0
CRASH_MONTH                    0
LIC_PLATE_STATE            12230
VEHICLE_TYPE                   0
MANEUVER                       0
FIRST_CONTACT_POINT           86
PERSON_TYPE                    0
SEX                         7393
AGE                        53949
SAFETY_EQUIPMENT            1633
AIRBAG_DEPLOYED            11910
DRIVER_ACT

# Andy's Space

In [147]:
defect_dict = {"NO DEFECTS" : "NO DEFECTS", "UNKNOWN": "NO DEFECTS", "WORN SURFACE": "DEFECTS", "OTHER":"DEFECTS","RUT, HOLES": "DEFECTS", "SHOULDER DEFECT":"DEFECTS", "DEBRIS ON ROADWAY": "DEFECTS"}

master_df['ROAD_DEFECT'] = master_df['ROAD_DEFECT'].map(defect_dict)
master_df['ROAD_DEFECT'].value_counts()

NO DEFECTS    301095
DEFECTS         4503
Name: ROAD_DEFECT, dtype: int64

In [148]:
master_df["CRASH_TYPE"].value_counts()

INJURY AND / OR TOW DUE TO CRASH    305506
NO INJURY / DRIVE AWAY                  92
Name: CRASH_TYPE, dtype: int64

In [149]:
master_df["DAMAGE"].value_counts()
master_df['DAMAGE'] = master_df['DAMAGE'].apply(lambda x: 'OVER $1500' if 'OVER $1,500' in x else "UNDER $1500")
master_df['DAMAGE'].value_counts()

OVER $1500     259341
UNDER $1500     46257
Name: DAMAGE, dtype: int64

In [150]:
#talk about this with team
master_df["PRIM_CONTRIBUTORY_CAUSE"].value_counts()

UNABLE TO DETERMINE                                                                 83621
FAILING TO YIELD RIGHT-OF-WAY                                                       53085
FOLLOWING TOO CLOSELY                                                               26314
FAILING TO REDUCE SPEED TO AVOID CRASH                                              26078
DISREGARDING TRAFFIC SIGNALS                                                        24689
IMPROPER TURNING/NO SIGNAL                                                          11340
IMPROPER OVERTAKING/PASSING                                                          9315
NOT APPLICABLE                                                                       9116
DISREGARDING STOP SIGN                                                               8654
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER     7296
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  6699
IMPROPER L

In [151]:
prim_cause_dict = { 'FOLLOWING TOO CLOSELY': "HUMAN FACTORS", 'FAILING TO REDUCE SPEED TO AVOID CRASH': "HUMAN FACTORS", 
                   'IMPROPER TURNING/NO SIGNAL': 'HUMAN FACTORS', 'IMPROPER OVERTAKING/PASSING' : 'HUMAN FACTORS',
                   'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER': 'HUMAN FACTORS',
                   'PHYSICAL CONDITION OF DRIVER': "HUMAN FACTORS", 
                   'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)': "HUMAN FACTORS",
                   'DISTRACTION - FROM INSIDE VEHICLE':"HUMAN FACTORS", 'DISTRACTION - FROM OUTSIDE VEHICLE': 'HUMAN FACTORS',
                   'TEXTING':"HUMAN FACTORS", 'IMPROPER BACKING':"HUMAN FACTORS", 'CELL PHONE USE OTHER THAN TEXTING': "HUMAN FACTORS",
                   'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)': "HUMAN FACTORS", 
                   'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)':"HUMAN FACTORS",
                   'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)': "ENVIRONMENTAL FACTORS", 'ANIMAL': "ENVIRONMENTAL FACTORS",
                   'WEATHER': "ENVIRONMENTAL FACTORS", 'EVASIVE ACTION DUE TO ANIMAL,OBJECT, NONMOTORIST': "ENVIRONMENTAL FACTORS",
                   'FAILING TO YIELD RIGHT-OF-WAY':"SIGNAGE/TRAFFIC CONTROL DEVICES", 'DISREGARDING TRAFFIC SIGNALS': "SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'DISREGARDING STOP SIGN':"SIGNAGE/TRAFFIC CONTROL DEVICES", 'DRIVING ON WRONG SIDE/WRONG WAY':"SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'IMPROPER LANE USAGE': "SIGNAGE/TRAFFIC CONTROL DEVICES", 'DISREGARDING OTHER TRAFFIC SIGNS':"SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'TURNING RIGHT ON RED': "SIGNAGE/TRAFFIC CONTROL DEVICES", 'DISREGARDING ROAD MARKINGS':"SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'DISREGARDING YIELD SIGN': "SIGNAGE/TRAFFIC CONTROL DEVICES", 'RELATED TO BUS STOP': "SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'PASSING STOPPED SCHOOL BUS': "SIGNAGE/TRAFFIC CONTROL DEVICES", 
                   'ROAD CONSTRUCTION/MAINTENANCE': "INFRASTRUCTURE ISSUES", 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS': "INFRASTRUCTURE ISSUES",
                   'OBSTRUCTED CROSSWALKS': "INFRASTRUCTURE ISSUES", 
                   'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE': "MISC FACTORS", 'BICYCLE ADVANCING LEGALLY ON RED LIGHT':"MISC FACTORS",
                   'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT':"MISC FACTORS", 'EQUIPMENT - VEHICLE CONDITION': "MISC FACTORS",
                   'NOT APPLICABLE': "MISC FACTORS"
              }

master_df['PRIM_CONTRIBUTORY_CAUSE'] = master_df['PRIM_CONTRIBUTORY_CAUSE'].map(prim_cause_dict)
master_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

SIGNAGE/TRAFFIC CONTROL DEVICES    100090
HUMAN FACTORS                       95907
MISC FACTORS                        18567
ENVIRONMENTAL FACTORS                6333
INFRASTRUCTURE ISSUES                 492
Name: PRIM_CONTRIBUTORY_CAUSE, dtype: int64

<details>
  <summary>**Click Me to Expand Breakdown of PRIM_CONTRIBUTORY_CAUSE**</summary>
    
  **human factors:**
    - 'FOLLOWING TOO CLOSELY', 
    - 'FAILING TO REDUCE SPEED TO AVOID CRASH',
    - 'IMPROPER TURNING/NO SIGNAL',
    - 'IMPROPER OVERTAKING/PASSING', 
    - 'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
    - 'PHYSICAL CONDITION OF DRIVER',
    - 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)',
    - 'DISTRACTION - FROM INSIDE VEHICLE',
    - 'DISTRACTION - FROM OUTSIDE VEHICLE',
    - 'TEXTING',
    - 'IMPROPER BACKING',
    - 'CELL PHONE USE OTHER THAN TEXTING',
    - 'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)',
    - 'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)',

**environmental factors:** 
    - 'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)',
    - 'ANIMAL', 
    - 'WEATHER',
    - 'EVASIVE ACTION DUE TO ANIMAL,OBJECT, NONMOTORIST',

**traffic control devices/signage:**
    - 'FAILING TO YIELD RIGHT-OF-WAY',
    - 'DISREGARDING TRAFFIC SIGNALS',
    - 'DISREGARDING STOP SIGN',
    - 'DRIVING ON WRONG SIDE/WRONG WAY',
    - 'IMPROPER LANE USAGE',
    - 'DISREGARDING OTHER TRAFFIC SIGNS',
    - 'TURNING RIGHT ON RED',
    - 'DISREGARDING ROAD MARKINGS',
    - 'DISREGARDING YIELD SIGN',
    - 'RELATED TO BUS STOP', 
    - 'PASSING STOPPED SCHOOL BUS',

**infrastructure issues:**
    - 'ROAD CONSTRUCTION/MAINTENANCE', 
    - 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS',
    - 'OBSTRUCTED CROSSWALKS',

**misc factors:**
    - 'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE', 
    - 'BICYCLE ADVANCING LEGALLY ON RED LIGHT', 
    - 'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT'
    - 'EQUIPMENT - VEHICLE CONDITION',
- 'NOT APPLICABLE'

--
'UNABLE TO DETERMINE', - to be dropped

</details>

In [152]:
master_df["NUM_UNITS"].value_counts()
#in int form already

2     202015
3      61190
4      22690
5       7534
1       5671
6       3596
7       1409
8       1014
9        291
12       131
11        53
14         4
Name: NUM_UNITS, dtype: int64

In [153]:
#change to no injury, minor, major, fatality
master_df["MOST_SEVERE_INJURY"].value_counts()

injury_dict = {'NONINCAPACITATING INJURY': 'MINOR INJURIES',
              'REPORTED, NOT EVIDENT': 'MINOR INJURIES', 'INCAPACITATING INJURY' : 'MAJOR INJURIES',
              'FATAL': 'FATAL'}

master_df['SEVERE_INJURY'] = master_df['MOST_SEVERE_INJURY'].map(injury_dict)
master_df.drop(labels="MOST_SEVERE_INJURY", axis=1, inplace=True)
master_df['SEVERE_INJURY'].value_counts()

MINOR INJURIES    267484
MAJOR INJURIES     35384
FATAL               2730
Name: SEVERE_INJURY, dtype: int64

In [154]:
master_df["INJURIES_TOTAL"].value_counts()
master_df["CRASH_HOUR"].value_counts()
master_df["CRASH_DAY_OF_WEEK"].value_counts()
master_df["CRASH_MONTH"].value_counts()
#int form already

7     32695
6     30152
8     29755
10    29626
9     29217
5     28983
4     23074
3     22853
1     22210
2     20199
11    19574
12    17260
Name: CRASH_MONTH, dtype: int64

In [155]:
master_df["LIC_PLATE_STATE"].value_counts()
master_df.drop(labels="LIC_PLATE_STATE", axis=1, inplace=True)
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305598 entries, 0 to 3594374
Data columns (total 34 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          305598 non-null  object        
 1   CRASH_DATE               305598 non-null  datetime64[ns]
 2   POSTED_SPEED_LIMIT       305598 non-null  int64         
 3   TRAFFIC_CONTROL_DEVICE   305598 non-null  object        
 4   DEVICE_CONDITION         305598 non-null  object        
 5   WEATHER_CONDITION        305598 non-null  object        
 6   LIGHTING_CONDITION       305598 non-null  object        
 7   FIRST_CRASH_TYPE         305598 non-null  object        
 8   TRAFFICWAY_TYPE          305598 non-null  object        
 9   ALIGNMENT                305598 non-null  object        
 10  ROADWAY_SURFACE_COND     305598 non-null  object        
 11  ROAD_DEFECT              305598 non-null  object        
 12  CRASH_TYPE     

In [156]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305598 entries, 0 to 3594374
Data columns (total 34 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          305598 non-null  object        
 1   CRASH_DATE               305598 non-null  datetime64[ns]
 2   POSTED_SPEED_LIMIT       305598 non-null  int64         
 3   TRAFFIC_CONTROL_DEVICE   305598 non-null  object        
 4   DEVICE_CONDITION         305598 non-null  object        
 5   WEATHER_CONDITION        305598 non-null  object        
 6   LIGHTING_CONDITION       305598 non-null  object        
 7   FIRST_CRASH_TYPE         305598 non-null  object        
 8   TRAFFICWAY_TYPE          305598 non-null  object        
 9   ALIGNMENT                305598 non-null  object        
 10  ROADWAY_SURFACE_COND     305598 non-null  object        
 11  ROAD_DEFECT              305598 non-null  object        
 12  CRASH_TYPE     

# Harshitha's Space

In [157]:
# DROP unnecessary columns
# master_df.drop(columns=['CRASH_RECORD_ID', 'CRASH_DATE'], inplace=True)

In [158]:
master_df["YEAR"].value_counts()

2021    81665
2022    78600
2023    74362
2020    70971
Name: YEAR, dtype: int64

In [159]:
master_df["POSTED_SPEED_LIMIT"].value_counts()

30    242017
35     27154
25     14365
20      6705
40      5834
45      3531
15      3446
10      1542
55       353
5        220
0        139
50       115
3         60
39        46
60        31
26        16
7          8
9          4
33         4
24         2
29         2
34         2
2          2
Name: POSTED_SPEED_LIMIT, dtype: int64

In [160]:
# can bin this into binary, Yes or no
master_df["TRAFFIC_CONTROL_DEVICE"].value_counts()

TRAFFIC SIGNAL              130369
NO CONTROLS                 115428
STOP SIGN/FLASHER            47894
UNKNOWN                       6868
OTHER                         1955
YIELD                          725
PEDESTRIAN CROSSING SIGN       501
FLASHING CONTROL SIGNAL        357
DELINEATORS                    311
SCHOOL ZONE                    262
RAILROAD CROSSING GATE         259
OTHER REG. SIGN                243
OTHER WARNING SIGN             169
POLICE/FLAGMAN                 108
OTHER RAILROAD CROSSING         81
RR CROSSING SIGN                41
BICYCLE CROSSING SIGN           19
NO PASSING                       8
Name: TRAFFIC_CONTROL_DEVICE, dtype: int64

In [161]:
master_df['TRAFFIC_CONTROL_DEVICE'] = master_df['TRAFFIC_CONTROL_DEVICE'].apply(lambda x: 'SIGNAL' if 'SIGNAL' in x else x)
master_df['TRAFFIC_CONTROL_DEVICE'].value_counts()

SIGNAL                      130726
NO CONTROLS                 115428
STOP SIGN/FLASHER            47894
UNKNOWN                       6868
OTHER                         1955
YIELD                          725
PEDESTRIAN CROSSING SIGN       501
DELINEATORS                    311
SCHOOL ZONE                    262
RAILROAD CROSSING GATE         259
OTHER REG. SIGN                243
OTHER WARNING SIGN             169
POLICE/FLAGMAN                 108
OTHER RAILROAD CROSSING         81
RR CROSSING SIGN                41
BICYCLE CROSSING SIGN           19
NO PASSING                       8
Name: TRAFFIC_CONTROL_DEVICE, dtype: int64

In [162]:
master_df['TRAFFIC_CONTROL_DEVICE'] = master_df['TRAFFIC_CONTROL_DEVICE'].apply(lambda x: 'SIGN' if ' SIGN' in x else x)
master_df['TRAFFIC_CONTROL_DEVICE'].value_counts()

SIGNAL                     130726
NO CONTROLS                115428
SIGN                        48867
UNKNOWN                      6868
OTHER                        1955
YIELD                         725
DELINEATORS                   311
SCHOOL ZONE                   262
RAILROAD CROSSING GATE        259
POLICE/FLAGMAN                108
OTHER RAILROAD CROSSING        81
NO PASSING                      8
Name: TRAFFIC_CONTROL_DEVICE, dtype: int64

In [163]:
traffic_dict = {'NO CONTROLS': 'NO CONTROLS', 'SIGNAL':'SIGNAL', 'SIGN': 'SIGN', 'UNKNOWN': 'OTHER',
               'OTHER': 'OTHER', 'LANE USE MARKING': 'SIGN', 'YIELD':'SIGN', 'RAILROAD CROSSING GATE': 'SIGNAL',
               'SCHOOL ZONE': 'SIGN', 'DELINEATORS': 'OTHER', 'POLICE/FLAGMAN': 'OTHER', 
               'OTHER RAILROAD CROSSING': 'SIGNAL', 'NO PASSING': 'SIGN', }
master_df['TRAFFIC_CONTROL_DEVICE'] = master_df['TRAFFIC_CONTROL_DEVICE'].map(traffic_dict)
master_df['TRAFFIC_CONTROL_DEVICE'].value_counts()

SIGNAL         131066
NO CONTROLS    115428
SIGN            49862
OTHER            9242
Name: TRAFFIC_CONTROL_DEVICE, dtype: int64

In [164]:
master_df['DEVICE_CONDITION'].value_counts()

FUNCTIONING PROPERLY        163434
NO CONTROLS                 119932
UNKNOWN                      16368
OTHER                         2595
FUNCTIONING IMPROPERLY        1927
NOT FUNCTIONING               1194
WORN REFLECTIVE MATERIAL       100
MISSING                         48
Name: DEVICE_CONDITION, dtype: int64

In [165]:
device_dict = {'NO CONTROLS': 'NO CONTROLS',
               'FUNCTIONING PROPERLY': 'FUNCTIONING PROPERLY',
               'UNKNOWN': 'NO CONTROLS',
               'FUNCTIONING IMPROPERLY':'FUNCTIONING IMPROPERLY/ MISSING',
               'NOT FUNCTIONING':'FUNCTIONING IMPROPERLY/ MISSING',
               'WORN REFLECTIVE MATERIAL': 'FUNCTIONING PROPERLY',
               'MISSING': 'FUNCTIONING IMPROPERLY/ MISSING',
               'OTHER': 'FUNCTIONING PROPERLY'}
master_df['DEVICE_CONDITION'] = master_df['DEVICE_CONDITION'].map(device_dict)
master_df['DEVICE_CONDITION'].value_counts()

FUNCTIONING PROPERLY               166129
NO CONTROLS                        136300
FUNCTIONING IMPROPERLY/ MISSING      3169
Name: DEVICE_CONDITION, dtype: int64

In [166]:
# bin into clear or not clear? unknown = clear, other=clear 
master_df["WEATHER_CONDITION"].value_counts()

CLEAR                       249988
RAIN                         30571
CLOUDY/OVERCAST               8955
SNOW                          7895
UNKNOWN                       4641
OTHER                         1242
FREEZING RAIN/DRIZZLE         1181
BLOWING SNOW                   407
FOG/SMOKE/HAZE                 393
SLEET/HAIL                     262
SEVERE CROSS WIND GATE          61
BLOWING SAND, SOIL, DIRT         2
Name: WEATHER_CONDITION, dtype: int64

In [167]:
weather_dict = {'CLEAR': 'CLEAR', 'RAIN': 'INCLEMENT', 'CLOUDY/OVERCAST': 'INCLEMENT', 'SNOW': 'INCLEMENT', 'UNKNOWN': 'CLEAR', 'OTHER': 'CLEAR',
                'FREEZING RAIN/DRIZZLE': 'INCLEMENT', 'BLOWING SNOW':'INCLEMENT', 'FOG/SMOKE/HAZE': 'INCLEMENT', 'SLEET/HAIL': 'INCLEMENT', 'SEVERE CROSS WIND GATE': 'INCLEMENT',
                'BLOWING SAND, SOIL, DIRT': 'INCLEMENT'
               }
master_df['WEATHER_CONDITION'] = master_df['WEATHER_CONDITION'].map(weather_dict)
master_df['WEATHER_CONDITION'].value_counts()

CLEAR        255871
INCLEMENT     49727
Name: WEATHER_CONDITION, dtype: int64

In [168]:
master_df["LIGHTING_CONDITION"].value_counts()

DAYLIGHT                  187201
DARKNESS, LIGHTED ROAD     89446
DARKNESS                   11653
DUSK                        9415
DAWN                        5033
UNKNOWN                     2850
Name: LIGHTING_CONDITION, dtype: int64

In [169]:
lighting_dict = {'DAYLIGHT':'VISIBLE','DARKNESS, LIGHTED ROAD':'DIMINISHED VISIBILITY', 'DARKNESS':'DIMINISHED VISIBILITY',
                'DUSK':'DIMINISHED VISIBILITY', 'DAWN': 'DIMINISHED VISIBILITY', 'UNKNOWN': 'VISIBLE'}
master_df['LIGHTING_CONDITION'] = master_df['LIGHTING_CONDITION'].map(lighting_dict)
master_df['LIGHTING_CONDITION'].value_counts()

VISIBLE                  190051
DIMINISHED VISIBILITY    115547
Name: LIGHTING_CONDITION, dtype: int64

In [170]:
# Looking at the FIRST_CRASH_TYPE column 
master_df['FIRST_CRASH_TYPE'].value_counts()

REAR END                        79129
ANGLE                           74256
TURNING                         64269
SIDESWIPE SAME DIRECTION        22372
PARKED MOTOR VEHICLE            17638
PEDESTRIAN                      14657
HEAD ON                          8905
PEDALCYCLIST                     8648
FIXED OBJECT                     6189
SIDESWIPE OPPOSITE DIRECTION     4935
REAR TO FRONT                    1520
REAR TO SIDE                     1463
OTHER OBJECT                     1087
OVERTURNED                        222
OTHER NONCOLLISION                147
REAR TO REAR                      101
ANIMAL                             32
TRAIN                              28
Name: FIRST_CRASH_TYPE, dtype: int64

In [171]:
first_crash_dict = {'REAR END': 'REAR END', 'SIDESWIPE SAME DIRECTION': 'SIDE SWIPE', 'TURNING': 'TURNING', 'PARKED MOTOR VEHICLE': 'OBJECT', 'ANGLE': 'ANGLE', 'PEDESTRIAN': 'PERSON OR ANIMAL',
                    'SIDESWIPE OPPOSITE DIRECTION': 'SIDE SWIPE', 'FIXED OBJECT': 'OBJECT', 'PEDALCYCLIST': 'PERSON OR ANIMAL', 'HEAD ON': 'OTHER', 'REAR TO FRONT': 'OTHER',
                    'REAR TO SIDE': 'OTHER', 'OTHER OBJECT': 'OBJECT', 'REAR TO REAR': 'OTHER', 'OTHER NONCOLLISION': 'OTHER', 'ANIMAL': 'PERSON OR ANIMAL', 'OVERTURNED': 'OTHER', 'TRAIN': 'OTHER'
                    }
master_df['FIRST_CRASH_TYPE'] = master_df['FIRST_CRASH_TYPE'].map(first_crash_dict)
master_df['FIRST_CRASH_TYPE'].value_counts()

REAR END            79129
ANGLE               74256
TURNING             64269
SIDE SWIPE          27307
OBJECT              24914
PERSON OR ANIMAL    23337
OTHER               12386
Name: FIRST_CRASH_TYPE, dtype: int64

In [172]:
master_df['TRAFFICWAY_TYPE'].value_counts()

NOT DIVIDED                        116522
FOUR WAY                            66367
DIVIDED - W/MEDIAN (NOT RAISED)     48121
DIVIDED - W/MEDIAN BARRIER          22812
ONE-WAY                             17980
T-INTERSECTION                      10854
OTHER                                5286
PARKING LOT                          3756
CENTER TURN LANE                     2954
UNKNOWN INTERSECTION TYPE            2455
ALLEY                                1796
FIVE POINT, OR MORE                  1199
Y-INTERSECTION                       1197
TRAFFIC ROUTE                        1067
UNKNOWN                              1033
NOT REPORTED                          853
RAMP                                  662
DRIVEWAY                              432
L-INTERSECTION                        154
ROUNDABOUT                             98
Name: TRAFFICWAY_TYPE, dtype: int64

In [173]:
#  
trafficway_dict = {'NOT DIVIDED': 'NOT DIVIDED', 'DIVIDED - W/MEDIAN (NOT RAISED)': 'DIVIDED', 'ONE-WAY': 'OTHER', 'FOUR WAY': 'OTHER', 'DIVIDED - W/MEDIAN BARRIER': 'DIVIDED',
                    'PARKING LOT': 'OTHER', 'OTHER': 'OTHER', 'T-INTERSECTION': 'INTERSECTION', 'ALLEY': 'OTHER', 'UNKNOWN': 'NOT DIVIDED', 'CENTER TURN LANE': 'OTHER', 
                    'UNKNOWN INTERSECTION TYPE': 'INTERSECTION', 'RAMP': 'OTHER', 'DRIVEWAY': 'OTHER', 'FIVE POINT, OR MORE': 'OTHER', 'Y-INTERSECTION': 'INTERSECTION', 
                    'TRAFFIC ROUTE': 'OTHER', 'NOT REPORTED': 'OTHER', 'ROUNDABOUT': 'INTERSECTION', 'L-INTERSECTION': 'INTERSECTION'
               }
master_df['TRAFFICWAY_TYPE'] = master_df['TRAFFICWAY_TYPE'].map(trafficway_dict)
master_df['TRAFFICWAY_TYPE'].value_counts()

NOT DIVIDED     117555
OTHER           102352
DIVIDED          70933
INTERSECTION     14758
Name: TRAFFICWAY_TYPE, dtype: int64

In [174]:
# Street alignment at crash location, as determined by reporting officer. maybe drop?
alignment_dict = {'STRAIGHT AND LEVEL': 'STRAIGHT', 'STRAIGHT ON GRADE': 'STRAIGHT', 'CURVE, LEVEL': 'NOT STRAIGHT', 'STRAIGHT ON HILLCREST': 'STRAIGHT',
                    'CURVE ON GRADE': 'NOT STRAIGHT', 'CURVE ON HILLCREST': 'NOT STRAIGHT'
               }
master_df['ALIGNMENT'] = master_df['ALIGNMENT'].map(alignment_dict)
master_df['ALIGNMENT'].value_counts()

STRAIGHT        302355
NOT STRAIGHT      3243
Name: ALIGNMENT, dtype: int64

In [175]:
roadway_surface_dict = {'DRY': 'DRY', 'WET': 'NOT DRY', 'UNKNOWN': 'DRY', 'SNOW OR SLUSH': 'NOT DRY', 'ICE': 'NOT DRY', 
                'OTHER': 'OTHER', 'SAND, MUD, DIRT': 'NOT DRY'
               }
master_df['ROADWAY_SURFACE_COND'] = master_df['ROADWAY_SURFACE_COND'].map(roadway_surface_dict)
master_df['ROADWAY_SURFACE_COND'].value_counts()

DRY        249355
NOT DRY     55410
OTHER         833
Name: ROADWAY_SURFACE_COND, dtype: int64

# Dre's Space

In [176]:
## Vehicle Type

master_df['VEHICLE_TYPE'].value_counts()

PASSENGER                      238025
SPORT UTILITY VEHICLE (SUV)     51648
VAN/MINI-VAN                    15925
Name: VEHICLE_TYPE, dtype: int64

In [177]:
# Maneuver

MANEUVER_dict = {'STRAIGHT AHEAD': 'STRAIGHT', 'TURNING LEFT': 'OTHER', 'SLOW/STOP IN TRAFFIC': 'OTHER', 'PARKED': 'OTHER', 'TURNING RIGHT': 'OTHER', 'UNKNOWN/NA': 'STRAIGHT',
                 'OTHER': 'OTHER', 'PASSING/OVERTAKING': 'OTHER', 'CHANGING LANES': 'OTHER', 'U-TURN': 'OTHER', 'BACKING': 'OTHER', 'STARTING IN TRAFFIC': 'OTHER', 
                 'AVOIDING VEHICLES/OBJECTS': 'OTHER', 'ENTERING TRAFFIC LANE FROM PARKING': 'OTHER', 'SKIDDING/CONTROL LOSS': 'OTHER', 'DRIVING WRONG WAY': 'OTHER',
                 'MERGING': 'OTHER', 'ENTER FROM DRIVE/ALLEY': 'OTHER', 'PARKED IN TRAFFIC LANE': 'OTHER', 'NEGOTIATING A CURVE': 'OTHER', 'LEAVING TRAFFIC LANE TO PARK': 'OTHER',
                 'TURNING ON RED': 'OTHER', 'DISABLED': 'OTHER', 'DRIVERLESS': 'OTHER', 'DIVERGING': 'OTHER'}

master_df['MANEUVER'] = master_df['MANEUVER'].map(MANEUVER_dict)

In [178]:

master_df['MANEUVER'].value_counts()

STRAIGHT    194291
OTHER       111307
Name: MANEUVER, dtype: int64

In [179]:
# FIRST CONTACT POINT

master_df['FIRST_CONTACT_POINT'] = master_df['FIRST_CONTACT_POINT'].fillna('FRONT')

FIRST_CONTACT_POINT_dict = {'FRONT': 'FRONT', 'REAR': 'REAR', 'FRONT-LEFT-CORNER': 'FRONT', 'FRONT-RIGHT-CORNER': 'FRONT', 'SIDE-LEFT': 'SIDES', 'SIDE-RIGHT': 'SIDES', 
                            'REAR-LEFT-CORNER': 'REAR',
                            'REAR-RIGHT-CORNER':'REAR', 'SIDE-LEFT-REAR': 'SIDES', 'SIDE-LEFT-FRONT':'SIDES', 'SIDE-RIGHT-REAR':'SIDES', 'SIDE-RIGHT-FRONT': 'SIDES',
                            'TOTAL (ALL AREAS)': 'OTHER', 'NONE': 'NONE', 'OTHER': 'OTHER', 'UNDER CARRIAGE': 'OTHER', 'TOP': 'OTHER', 'ROOF': 'OTHER', 'UNKNOWN': 'FRONT',
                            'REAR-LEFT': 'REAR', 'FRONT-RIGHT': 'FRONT'}

master_df['FIRST_CONTACT_POINT'] = master_df['FIRST_CONTACT_POINT'].map(FIRST_CONTACT_POINT_dict)

In [180]:
master_df['FIRST_CONTACT_POINT'].value_counts(dropna=False)

FRONT    179686
REAR      63448
SIDES     57061
OTHER      4030
NONE       1373
Name: FIRST_CONTACT_POINT, dtype: int64

In [181]:
# Person Type

PERSON_TYPE_dict = {'DRIVER': 'DRIVER', 'PASSENGER': 'PASSENGER', 'PEDESTRIAN': 'OTHER', 'BICYCLE': 'OTHER', 'NON-MOTOR VEHICLE': 'OTHER', 'NON-CONTACT VEHICLE': 'OTHER'}

master_df['PERSON_TYPE'] = master_df['PERSON_TYPE'].map(PERSON_TYPE_dict)

In [182]:
master_df['PERSON_TYPE'].value_counts(dropna=False)

DRIVER       198143
PASSENGER     95401
OTHER         12054
Name: PERSON_TYPE, dtype: int64

In [183]:
# Sex

master_df = master_df[master_df['SEX'] != 'X']

master_df['SEX'].value_counts(dropna=False)

M      157656
F      129998
NaN      7393
Name: SEX, dtype: int64

In [184]:
# Age

master_df['AGE'].value_counts()

# Filter for ages 16-75??

25.0     7372
23.0     7232
27.0     7173
24.0     7132
28.0     7108
         ... 
101.0       4
110.0       2
108.0       2
97.0        2
105.0       1
Name: AGE, Length: 107, dtype: int64

In [185]:
# Safety Equipment

master_df['SAFETY_EQUIPMENT'] = master_df['SAFETY_EQUIPMENT'].fillna('USAGE UNKNOWN')

SAFETY_EQUIPMENT_dict = {'USAGE UNKNOWN': 'UNKNOWN', 'SAFETY BELT USED': 'USED', 'NONE PRESENT': 'NONE', 'HELMET NOT USED': 'NOT USED', 'SAFETY BELT NOT USED': 'NOT USED',
                         'CHILD RESTRAINT - FORWARD FACING': 'USED', 'BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)': 'USED', 'CHILD RESTRAINT - TYPE UNKNOWN': 'USED',
                         'CHILD RESTRAINT - REAR FACING': 'USED', 'DOT COMPLIANT MOTORCYCLE HELMET': 'USED', 'BOOSTER SEAT': 'USED', 'CHILD RESTRAINT NOT USED': 'NOT USED', 
                         'CHILD RESTRAINT USED IMPROPERLY': 'USED', 'NOT DOT COMPLIANT MOTORCYCLE HELMET': 'USED', 'WHEELCHAIR': 'USED', 'SHOULD/LAP BELT USED IMPROPERLY': 'USED',
                         'HELMET USED': 'USED', 'STRETCHER': 'USED', 'CHILD RESTRAINT USED': 'USED'}

master_df['SAFETY_EQUIPMENT'] = master_df['SAFETY_EQUIPMENT'].map(SAFETY_EQUIPMENT_dict)

In [186]:
master_df['SAFETY_EQUIPMENT'].value_counts(dropna=False)

UNKNOWN     139877
USED        136400
NONE         12606
NOT USED      6164
Name: SAFETY_EQUIPMENT, dtype: int64

In [187]:
# Airbag Deployed

master_df['AIRBAG_DEPLOYED'] = master_df['AIRBAG_DEPLOYED'].fillna('DID NOT DEPLOY')

AIRBAG_DEPLOYED_dict = {'DID NOT DEPLOY': 'DID NOT DEPLOY', 'NOT APPLICABLE': 'NOT APPLICABLE', 'DEPLOYED, COMBINATION': 'DEPLOYED', 'DEPLOYMENT UNKNOWN': 'UNKNOWN', 
                        'DEPLOYED, FRONT': 'DEPLOYED', 'DEPLOYED, SIDE': 'DEPLOYED', 'DEPLOYED OTHER (KNEE, AIR, BELT, ETC.)': 'DEPLOYED'}

master_df['AIRBAG_DEPLOYED'] = master_df['AIRBAG_DEPLOYED'].map(AIRBAG_DEPLOYED_dict)

In [188]:
master_df['AIRBAG_DEPLOYED'].value_counts(dropna=False)

DID NOT DEPLOY    115041
DEPLOYED           83943
NOT APPLICABLE     65199
UNKNOWN            30864
Name: AIRBAG_DEPLOYED, dtype: int64

In [189]:
# Driver Action

master_df['DRIVER_ACTION'] = master_df['DRIVER_ACTION'].fillna('NONE')

DRIVER_ACTION_dict = {'NONE': 'NO ACTION TAKEN', 'UNKNOWN': 'NO ACTION TAKEN', 'FAILED TO YIELD': 'ACTIONS TAKEN', 'OTHER': 'ACTIONS TAKEN', 'DISREGARDED CONTROL DEVICES': 'ACTIONS TAKEN',
                      'FOLLOWED TOO CLOSELY': 'ACTIONS TAKEN', 'TOO FAST FOR CONDITIONS': 'ACTIONS TAKEN', 'IMPROPER TURN': 'ACTIONS TAKEN', 'IMPROPER LANE CHANGE': 'ACTIONS TAKEN',
                      'IMPROPER PASSING': 'ACTIONS TAKEN', 'WRONG WAY/SIDE': 'ACTIONS TAKEN', 'IMPROPER BACKING': 'ACTIONS TAKEN', 'OVERCORRECTED': 'ACTIONS TAKEN', 'EVADING POLICE VEHICLE': 'ACTIONS TAKEN',
                      'CELL PHONE USE OTHER THAN TEXTING': 'ACTIONS TAKEN', 'EMERGENCY VEHICLE ON CALL': 'ACTIONS TAKEN', 'IMPROPER PARKING': 'ACTIONS TAKEN', 'TEXTING': 'ACTIONS TAKEN',
                      'LICENSE RESTRICTIONS': 'ACTIONS TAKEN', 'STOPPED SCHOOL BUS': 'ACTIONS TAKEN'}

master_df['DRIVER_ACTION'] = master_df['DRIVER_ACTION'].map(DRIVER_ACTION_dict)

In [190]:
master_df['DRIVER_ACTION'].value_counts(dropna=False)

NO ACTION TAKEN    216815
ACTIONS TAKEN       78232
Name: DRIVER_ACTION, dtype: int64

In [191]:
# Driver Vision

master_df['DRIVER_VISION'] = master_df['DRIVER_VISION'].fillna('NONE')

DRIVER_VISION_dict = { 'NONE':'UNKNOWN', 'NOT OBSCURED': 'NOT OBSCURED', 'UNKNOWN': 'UNKNOWN', 'OTHER': 'OBSCURED', 'MOVING VEHICLES': 'OBSCURED', 'PARKED VEHICLES': 'OBSCURED',
                      'WINDSHIELD (WATER/ICE)': 'OBSCURED', 'BLINDED - SUNLIGHT': 'OBSCURED', 'TREES, PLANTS': 'OBSCURED', 'BUILDINGS': 'OBSCURED',
                      'BLINDED - HEADLIGHTS': 'OBSCURED', 'BLOWING MATERIALS': 'OBSCURED', 'HILLCREST': 'OBSCURED',
                      'EMBANKMENT': 'OBSCURED', 'SIGNBOARD': 'OBSCURED'}

master_df['DRIVER_VISION'] = master_df['DRIVER_VISION'].map(DRIVER_VISION_dict)

In [192]:
master_df['DRIVER_VISION'].value_counts(dropna=False)

UNKNOWN         182906
NOT OBSCURED    105706
OBSCURED          6435
Name: DRIVER_VISION, dtype: int64

In [193]:
# Physical Condition

master_df['PHYSICAL_CONDITION'] = master_df['PHYSICAL_CONDITION'].fillna('UNKNOWN')

PHYSICAL_CONDITION_dict = {'UNKNOWN': 'UNKNOWN', 'NORMAL': 'NORMAL', 'REMOVED BY EMS': 'NOT NORMAL', 'IMPAIRED - ALCOHOL': 'NOT NORMAL', 'OTHER': 'NOT NORMAL', 'EMOTIONAL': 'NOT NORMAL',
                           'FATIGUED/ASLEEP': 'NOT NORMAL', 'ILLNESS/FAINTED': 'NOT NORMAL', 'IMPAIRED - DRUGS': 'NOT NORMAL', 'HAD BEEN DRINKING': 'NOT NORMAL', 
                           'IMPAIRED - ALCOHOL AND DRUGS': 'NOT NORMAL', 'MEDICATED': 'NOT NORMAL'}

master_df['PHYSICAL_CONDITION'] = master_df['PHYSICAL_CONDITION'].map(PHYSICAL_CONDITION_dict)

In [194]:
master_df['PHYSICAL_CONDITION'].value_counts(dropna=False)

NORMAL        146803
UNKNOWN       137339
NOT NORMAL     10905
Name: PHYSICAL_CONDITION, dtype: int64

In [195]:
# BAC Result

master_df['BAC_RESULT'] = master_df['BAC_RESULT'].fillna('UNKNOWN')

BAC_RESULT_dict = {'TEST NOT OFFERED': 'TEST NOT OFFERED', 'TEST REFUSED': 'TEST REFUSED', 'TEST PERFORMED, RESULTS UNKNOWN': 'TEST PERFORMED',
                   'TEST TAKEN': 'TEST PERFORMED'}

master_df['BAC_RESULT'] = master_df['BAC_RESULT'].map(BAC_RESULT_dict)

In [196]:
master_df['BAC_RESULT'].value_counts()

TEST NOT OFFERED    195341
TEST REFUSED          2702
TEST PERFORMED        1955
Name: BAC_RESULT, dtype: int64

## Preprocessing for Modeling

In [197]:
#View our cleaned columns
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295047 entries, 0 to 3594374
Data columns (total 34 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          295047 non-null  object        
 1   CRASH_DATE               295047 non-null  datetime64[ns]
 2   POSTED_SPEED_LIMIT       295047 non-null  int64         
 3   TRAFFIC_CONTROL_DEVICE   295047 non-null  object        
 4   DEVICE_CONDITION         295047 non-null  object        
 5   WEATHER_CONDITION        295047 non-null  object        
 6   LIGHTING_CONDITION       295047 non-null  object        
 7   FIRST_CRASH_TYPE         295047 non-null  object        
 8   TRAFFICWAY_TYPE          295047 non-null  object        
 9   ALIGNMENT                295047 non-null  object        
 10  ROADWAY_SURFACE_COND     295047 non-null  object        
 11  ROAD_DEFECT              295047 non-null  object        
 12  CRASH_TYPE     

In [198]:
#We can get rid of RECORD_ID and DATE, not relevant for ML
master_df.drop(labels=["CRASH_RECORD_ID","CRASH_DATE"], axis=1, inplace=True)

In [245]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295047 entries, 0 to 3594374
Data columns (total 32 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   POSTED_SPEED_LIMIT       295047 non-null  int64  
 1   TRAFFIC_CONTROL_DEVICE   295047 non-null  object 
 2   DEVICE_CONDITION         295047 non-null  object 
 3   WEATHER_CONDITION        295047 non-null  object 
 4   LIGHTING_CONDITION       295047 non-null  object 
 5   FIRST_CRASH_TYPE         295047 non-null  object 
 6   TRAFFICWAY_TYPE          295047 non-null  object 
 7   ALIGNMENT                295047 non-null  object 
 8   ROADWAY_SURFACE_COND     295047 non-null  object 
 9   ROAD_DEFECT              295047 non-null  object 
 10  CRASH_TYPE               295047 non-null  object 
 11  DAMAGE                   295047 non-null  object 
 12  PRIM_CONTRIBUTORY_CAUSE  214640 non-null  object 
 13  NUM_UNITS                295047 non-null  int64  
 14  INJ

In [264]:
#Set our target
y = master_df["PRIM_CONTRIBUTORY_CAUSE"]
X = master_df.drop("PRIM_CONTRIBUTORY_CAUSE", axis=1)

In [265]:
#Perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

In [266]:
#Check for any missing values left
X_train.isna().sum()

POSTED_SPEED_LIMIT            0
TRAFFIC_CONTROL_DEVICE        0
DEVICE_CONDITION              0
WEATHER_CONDITION             0
LIGHTING_CONDITION            0
FIRST_CRASH_TYPE              0
TRAFFICWAY_TYPE               0
ALIGNMENT                     0
ROADWAY_SURFACE_COND          0
ROAD_DEFECT                   0
CRASH_TYPE                    0
DAMAGE                        0
NUM_UNITS                     0
INJURIES_TOTAL                0
CRASH_HOUR                    0
CRASH_DAY_OF_WEEK             0
CRASH_MONTH                   0
VEHICLE_TYPE                  0
MANEUVER                      0
FIRST_CONTACT_POINT           0
PERSON_TYPE                   0
SEX                        5542
AGE                       32862
SAFETY_EQUIPMENT              0
AIRBAG_DEPLOYED               0
DRIVER_ACTION                 0
DRIVER_VISION                 0
PHYSICAL_CONDITION            0
BAC_RESULT                71174
YEAR                          0
SEVERE_INJURY                 0
dtype: i

In [267]:
#SEX - drop missing values
X_train.dropna(subset=["SEX"], inplace=True)

<ipython-input-267-a9aec4c826ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.dropna(subset=["SEX"], inplace=True)


In [268]:
X_train.isna().sum()

POSTED_SPEED_LIMIT            0
TRAFFIC_CONTROL_DEVICE        0
DEVICE_CONDITION              0
WEATHER_CONDITION             0
LIGHTING_CONDITION            0
FIRST_CRASH_TYPE              0
TRAFFICWAY_TYPE               0
ALIGNMENT                     0
ROADWAY_SURFACE_COND          0
ROAD_DEFECT                   0
CRASH_TYPE                    0
DAMAGE                        0
NUM_UNITS                     0
INJURIES_TOTAL                0
CRASH_HOUR                    0
CRASH_DAY_OF_WEEK             0
CRASH_MONTH                   0
VEHICLE_TYPE                  0
MANEUVER                      0
FIRST_CONTACT_POINT           0
PERSON_TYPE                   0
SEX                           0
AGE                       28414
SAFETY_EQUIPMENT              0
AIRBAG_DEPLOYED               0
DRIVER_ACTION                 0
DRIVER_VISION                 0
PHYSICAL_CONDITION            0
BAC_RESULT                65719
YEAR                          0
SEVERE_INJURY                 0
dtype: i

In [269]:
#age 
X_train["AGE"].value_counts()

 25.0     5499
 23.0     5418
 27.0     5358
 24.0     5354
 28.0     5336
          ... 
-1.0         5
 100.0       4
 101.0       3
 110.0       2
 105.0       1
Name: AGE, Length: 105, dtype: int64

In [270]:
X_train["AGE"]=X_train['AGE'].fillna(X_train['AGE'].mean())
X_train['AGE'].value_counts()

<ipython-input-270-c591d441418d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["AGE"]=X_train['AGE'].fillna(X_train['AGE'].mean())


 35.598172     28414
 25.000000      5499
 23.000000      5418
 27.000000      5358
 24.000000      5354
               ...  
-1.000000          5
 100.000000        4
 101.000000        3
 110.000000        2
 105.000000        1
Name: AGE, Length: 106, dtype: int64

In [271]:
X_train['BAC_RESULT'].fillna(value="TEST NOT OFFERED",axis=0, inplace=True)
X_train['BAC_RESULT'].value_counts()

D:\Documents\anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


TEST NOT OFFERED    212220
TEST REFUSED          2032
TEST PERFORMED        1491
Name: BAC_RESULT, dtype: int64

In [272]:
X_train.info()
X_train.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215743 entries, 1316341 to 702273
Data columns (total 31 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   POSTED_SPEED_LIMIT      215743 non-null  int64  
 1   TRAFFIC_CONTROL_DEVICE  215743 non-null  object 
 2   DEVICE_CONDITION        215743 non-null  object 
 3   WEATHER_CONDITION       215743 non-null  object 
 4   LIGHTING_CONDITION      215743 non-null  object 
 5   FIRST_CRASH_TYPE        215743 non-null  object 
 6   TRAFFICWAY_TYPE         215743 non-null  object 
 7   ALIGNMENT               215743 non-null  object 
 8   ROADWAY_SURFACE_COND    215743 non-null  object 
 9   ROAD_DEFECT             215743 non-null  object 
 10  CRASH_TYPE              215743 non-null  object 
 11  DAMAGE                  215743 non-null  object 
 12  NUM_UNITS               215743 non-null  int64  
 13  INJURIES_TOTAL          215743 non-null  float64
 14  CRASH_HOUR    

POSTED_SPEED_LIMIT        0
TRAFFIC_CONTROL_DEVICE    0
DEVICE_CONDITION          0
WEATHER_CONDITION         0
LIGHTING_CONDITION        0
FIRST_CRASH_TYPE          0
TRAFFICWAY_TYPE           0
ALIGNMENT                 0
ROADWAY_SURFACE_COND      0
ROAD_DEFECT               0
CRASH_TYPE                0
DAMAGE                    0
NUM_UNITS                 0
INJURIES_TOTAL            0
CRASH_HOUR                0
CRASH_DAY_OF_WEEK         0
CRASH_MONTH               0
VEHICLE_TYPE              0
MANEUVER                  0
FIRST_CONTACT_POINT       0
PERSON_TYPE               0
SEX                       0
AGE                       0
SAFETY_EQUIPMENT          0
AIRBAG_DEPLOYED           0
DRIVER_ACTION             0
DRIVER_VISION             0
PHYSICAL_CONDITION        0
BAC_RESULT                0
YEAR                      0
SEVERE_INJURY             0
dtype: int64

In [274]:
##list of columns to be encoded:
category_list = ['TRAFFIC_CONTROL_DEVICE','DEVICE_CONDITION',"WEATHER_CONDITION","LIGHTING_CONDITION","FIRST_CRASH_TYPE","TRAFFICWAY_TYPE",
                 "ALIGNMENT","ROADWAY_SURFACE_COND","ROAD_DEFECT","CRASH_TYPE","DAMAGE","VEHICLE_TYPE","MANEUVER","FIRST_CONTACT_POINT","PERSON_TYPE","SEX",
                "SAFETY_EQUIPMENT","AIRBAG_DEPLOYED","DRIVER_ACTION","DRIVER_VISION","PHYSICAL_CONDITION","BAC_RESULT","SEVERE_INJURY"]

In [275]:
#We will One-hot encode all of our columns
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories='auto')

encoded_train_arr = ohe.fit_transform(X_train[category_list]).toarray()
encoded_train_labels = ohe.get_feature_names()
encoded_train_labels
encoded_train_labels = np.array(encoded_train_labels).ravel()
encoded_train_arr = pd.DataFrame(encoded_train_arr, columns = encoded_train_labels, index=X_train.index)
encoded_train_arr

,x0_NO CONTROLS,x0_OTHER,x0_SIGN,x0_SIGNAL,x1_FUNCTIONING IMPROPERLY/ MISSING,x1_FUNCTIONING PROPERLY,x1_NO CONTROLS,x2_CLEAR,x2_INCLEMENT,x3_DIMINISHED VISIBILITY,...,x19_UNKNOWN,x20_NORMAL,x20_NOT NORMAL,x20_UNKNOWN,x21_TEST NOT OFFERED,x21_TEST PERFORMED,x21_TEST REFUSED,x22_FATAL,x22_MAJOR INJURIES,x22_MINOR INJURIES
1316341,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1292667,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2171485,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3402319,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3187369,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2642959,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
821396,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3151933,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2618407,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [276]:
#Drop the original columns and append our encoded array
X_train.drop(category_list, axis=1, inplace=True)
X_train = pd.concat([X_train, encoded_train_arr], axis=1)

D:\Documents\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,POSTED_SPEED_LIMIT,NUM_UNITS,INJURIES_TOTAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,AGE,YEAR
1316341,30,2,1.0,13,7,6,35.598172,2020
1292667,30,2,1.0,18,3,9,50.000000,2020
2171485,30,2,1.0,7,4,1,78.000000,2021
3402319,30,2,1.0,15,4,9,2.000000,2021
3187369,30,2,1.0,17,3,5,27.000000,2020
...,...,...,...,...,...,...,...,...
2642959,30,2,2.0,23,4,12,35.598172,2021
821396,30,5,1.0,18,2,12,49.000000,2022
3151933,30,2,1.0,19,6,12,67.000000,2020
2618407,35,2,1.0,9,3,8,67.000000,2023


In [277]:
# Visually inspect X_train
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215743 entries, 1316341 to 702273
Data columns (total 78 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   POSTED_SPEED_LIMIT                   215743 non-null  int64  
 1   NUM_UNITS                            215743 non-null  int64  
 2   INJURIES_TOTAL                       215743 non-null  float64
 3   CRASH_HOUR                           215743 non-null  int64  
 4   CRASH_DAY_OF_WEEK                    215743 non-null  int64  
 5   CRASH_MONTH                          215743 non-null  int64  
 6   AGE                                  215743 non-null  float64
 7   YEAR                                 215743 non-null  int64  
 8   x0_NO CONTROLS                       215743 non-null  float64
 9   x0_OTHER                             215743 non-null  float64
 10  x0_SIGN                              215743 non-null  float64
 11  x0_SIGN

## Modeling: 

## **Testing:**

In [283]:
#Do the same changes to X_test:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73762 entries, 923368 to 1737261
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   POSTED_SPEED_LIMIT      73762 non-null  int64  
 1   TRAFFIC_CONTROL_DEVICE  73762 non-null  object 
 2   DEVICE_CONDITION        73762 non-null  object 
 3   WEATHER_CONDITION       73762 non-null  object 
 4   LIGHTING_CONDITION      73762 non-null  object 
 5   FIRST_CRASH_TYPE        73762 non-null  object 
 6   TRAFFICWAY_TYPE         73762 non-null  object 
 7   ALIGNMENT               73762 non-null  object 
 8   ROADWAY_SURFACE_COND    73762 non-null  object 
 9   ROAD_DEFECT             73762 non-null  object 
 10  CRASH_TYPE              73762 non-null  object 
 11  DAMAGE                  73762 non-null  object 
 12  NUM_UNITS               73762 non-null  int64  
 13  INJURIES_TOTAL          73762 non-null  float64
 14  CRASH_HOUR              73762 n

In [284]:
#deal with missing values
X_test.dropna(subset=["SEX"], inplace=True)
X_test["AGE"]=X_test['AGE'].fillna(X_test['AGE'].mean())
X_test['BAC_RESULT'].fillna(value="TEST NOT OFFERED",axis=0, inplace=True)

#encode the categoricals
encoded_test_arr = ohe.fit_transform(X_test[category_list]).toarray()
encoded_test_labels = ohe.get_feature_names()
encoded_test_labels
encoded_test_labels = np.array(encoded_test_labels).ravel()
encoded_test_arr = pd.DataFrame(encoded_test_arr, columns = encoded_test_labels, index=X_test.index)
encoded_test_arr

#drop orginal column and append our encoded columns
X_test.drop(category_list, axis=1, inplace=True)
X_test = pd.concat([X_test, encoded_test_arr], axis=1)

<ipython-input-284-167ce5bde942>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.dropna(subset=["SEX"], inplace=True)
<ipython-input-284-167ce5bde942>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["AGE"]=X_test['AGE'].fillna(X_test['AGE'].mean())
D:\Documents\anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [287]:
#inspect X_test
X_test.info()
X_test.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71911 entries, 923368 to 1737261
Data columns (total 78 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   POSTED_SPEED_LIMIT                   71911 non-null  int64  
 1   NUM_UNITS                            71911 non-null  int64  
 2   INJURIES_TOTAL                       71911 non-null  float64
 3   CRASH_HOUR                           71911 non-null  int64  
 4   CRASH_DAY_OF_WEEK                    71911 non-null  int64  
 5   CRASH_MONTH                          71911 non-null  int64  
 6   AGE                                  71911 non-null  float64
 7   YEAR                                 71911 non-null  int64  
 8   x0_NO CONTROLS                       71911 non-null  float64
 9   x0_OTHER                             71911 non-null  float64
 10  x0_SIGN                              71911 non-null  float64
 11  x0_SIGNAL            

POSTED_SPEED_LIMIT    0
NUM_UNITS             0
INJURIES_TOTAL        0
CRASH_HOUR            0
CRASH_DAY_OF_WEEK     0
                     ..
x21_TEST PERFORMED    0
x21_TEST REFUSED      0
x22_FATAL             0
x22_MAJOR INJURIES    0
x22_MINOR INJURIES    0
Length: 78, dtype: int64